# Интуиция доверительных интервалов.

Решил написать небольшой пост про интерпретацию доверительных интeрвалов. Представим что у нас есть какая-то нормально распределенная совокупность, например средние скорости автомобилей на определенном участке МКАД. Из этой совокупности была получена выборка размером n = 50 и средним $\bar{x} = 90$ км/ч. Так же известно, что стандартное отклонение совокупности $\sigma = 7$ км/ч. Давайте построим доверительный интервал для среднего по совокупности с доверительным уровнем 95%. Что для этого нужно?

#### 1) Определить значение z для $\alpha = 1 - 0.95$. Делается это так: берем стандартное нормальное распределение c $\mu = 0$ и $\sigma = 1$ и с помощью функции ppf (инверсия функции распределения cdf) считаем значение z.

In [1]:
from scipy import stats
import numpy as np
import math

In [2]:
alpha = 1 - 0.95
z = abs(stats.norm.ppf(alpha/2))
z

1.959963984540054

В принципе это стандартное значение и его часто округляют до 1.96, а иногда используют правило двух сигм и берут это значение равны просто 2.

#### 2) Оперделяем стандартную ошибку среднего $SE = \frac{\sigma}{\sqrt{n}}$

In [3]:
sigma = 7
n = 50
SE = sigma/math.sqrt(n)
SE

0.9899494936611665

#### 3) Рассчитать нижнюю и верхнуюю границы доверительного интервала. Они определяются как $\bar{x}\pm z*SE$.

In [4]:
xbar = 90
ci_low = xbar - z * SE # ci - сокращение от confidence interval
ci_hi = xbar + z * SE
(ci_low, ci_hi)

(88.059734645910453, 91.940265354089547)

Получилось, что 95% доверительный интервал для средней скорости составил (88.06, 91.94). Так же можно сказать, что есть 95% вероятность того, что истинное среднее по совокупности лежит в интервале от 88.06 до 91.94. Как это интерпретировать? Например так: если брать из нашей совокупности выборки по 50, находить среднее по выборке и строить на его основе интервалы по приведенной выше схеме, то только 95% из этих интервалов будут содержать истинное среднее по совокупности. Но зачем же верить мне на слово? У нас есть питон и мы можем убедиться в том, что доверительные интервалы работают именно так, с помощью экспериментов над случайными числами (надеюсь числа не пострадают)! Давайте сделаем это! Я напишу следующую функцию:

In [5]:
def test_conf_norm(dist,alpha,n,num_exp=10000):
    
    z = abs(stats.norm.ppf(alpha/2))
    mu = dist.mean()
    sigma = dist.std()
    
    result_norm = np.zeros(num_exp)
    
    for exp in range(num_exp):
    
        sample = dist.rvs(n)
        xbar = np.mean(sample)
        
        SE = sigma/math.sqrt(len(sample))
        
        ci_low_norm = xbar - z*SE
        ci_hi_norm = xbar + z*SE
        
        if ci_low_norm <= mu <= ci_hi_norm:
            result_norm[exp] = 1
        
    return result_norm.mean()

Эта функция берет на вход распределение dist (объект распределения из scipy.stats), alpha (1 - доверительный уровень), размер выборки n и количество экспериментов (по умолчанию это 10000). Функция строит 10000 доверительных интервалов и проверяет, столько из них содержит истинное среднее. На выходе она дает долю экспериментов, в которых доверительный интервал содержал истинное среднее. Давайте создадим наше распределение со средними скоростями и прогоним для него эту функцию.

In [6]:
mean_speed = stats.norm(loc=90,scale=7)
test_conf_norm(mean_speed,0.05,50)

0.94869999999999999

Проведя 10000 экспериментов, мы получили долю доверительных интервалов (с доверительным уровнем 95%) содержащих истинное среднее очень близкую к 0.95 - что и требовалось доказать. Кстати, при известном стандартном отклонении по нормально распределенной совокупности доверительные интервалы будут работать даже при очень маленьких выборках.

In [7]:
test_conf_norm(mean_speed,0.05,4)

0.95240000000000002

Но что если нам не известно стандартное отклонение по совокупноти? Ведь в реальных ситуациях у нас есть выборка, и больше мы ничего не знаем. В этом случае, если выборка достаточно большая (n >= 30) можно использовать стандартное отклонение по выборке вместо стандартно отклонения по совокупности. Давайте должным образом изменим первую функцию и проведем эксперементы.

In [8]:
def test_conf_norm_s(dist,alpha,n,num_exp=10000):
    
    z = abs(stats.norm.ppf(alpha/2))
    mu = dist.mean()
    
    result_norm = np.zeros(num_exp)
    
    for exp in range(num_exp):
    
        sample = dist.rvs(n)
        xbar = np.mean(sample)
        s = sample.std(ddof=1) # ddof=1 для несмещенной оценки
        
        SE = s/math.sqrt(len(sample)) # s вместо sigma
        
        ci_low_norm = xbar - z*SE
        ci_hi_norm = xbar + z*SE
        
        if ci_low_norm <= mu <= ci_hi_norm:
            result_norm[exp] = 1
        
    return result_norm.mean()

In [9]:
test_conf_norm_s(mean_speed,0.05,50)

0.94210000000000005

Как видим - все работает. Но что если выборка маленькая (n < 30)?

In [10]:
test_conf_norm_s(mean_speed,0.05,4)

0.85360000000000003

Не работает(. Что же делать? Нужно использовать распределение Стьюдента и вместо z-статистики считать t-статистику. Распределение Стьюдента задается одним параметром - количеством степеней свободы, которое равно n-1. Изменим нашу функцию соответствующим образом и проверим будут ли работать доверительные интревалы.

In [11]:
def test_conf_t_s(dist,alpha,n,num_exp=10000):
    
    t = abs(stats.t.ppf(alpha/2,n-1)) # t-статистика
    mu = dist.mean()
    
    result_norm = np.zeros(num_exp)
    
    for exp in range(num_exp):
    
        sample = dist.rvs(n)
        xbar = np.mean(sample)
        s = sample.std(ddof=1) # ddof=1 для несмещенной оценки
        
        SE = s/math.sqrt(len(sample)) # s вместо sigma
        
        ci_low_norm = xbar - t*SE # меняем z на t
        ci_hi_norm = xbar + t*SE
        
        if ci_low_norm <= mu <= ci_hi_norm:
            result_norm[exp] = 1
        
    return result_norm.mean()

In [12]:
test_conf_t_s(mean_speed,0.05,4)

0.95150000000000001

Вот! Другое дело! Вообще формально при неизвестном $\sigma$ совокупности мы должны всегда использовать t-статистику, но при n >= 30 t-распределение практически ни чем не отличается от z-распределения (стандартного нормального распределения). При построении доверительных интервалов можно руководствоваться следующими правилами:

| Условия | Доверительный интревал |         
| :- |:-------------: |
|n >= 30, $\sigma$ известно, любая совокупность|  $\bar{x}\pm z\frac{\sigma}{\sqrt{n}}$. |
|n >= 30, $\sigma$ неизвестно, любая совокупность|  $\bar{x}\pm t\frac{s}{\sqrt{n}}$. |
|n < 30, $\sigma$ известно, норм. совокупность|  $\bar{x}\pm z\frac{\sigma}{\sqrt{n}}$. |
|n < 30, $\sigma$ неизвестно, норм. совокупность|  $\bar{x}\pm t\frac{s}{\sqrt{n}}$. |

Теперь давайте рассмотрим доверительные интервалы для доли. Например представим, что мы опросили 50 владельцев iPhone 6s и 30 из них подтвердили, что собираются купить iPhone SE. Нужно построить 95% доверительный интервал для этой доли. Что будем делать?

#### 1) Рассчитаем долю $p = \frac{кол-во успехов}{n} $

In [13]:
p = 30/50.
p

0.6

#### 2) Рассчитаем z-статистику для уровня 95%.

In [14]:
alpha = 0.05
n = 50
z = abs(stats.norm.ppf(alpha/2))
z

1.9599639845400545

#### 3) Рассчитаем стандартную ошибку среднего $SE = \sqrt{\frac{pq}{n}} $, где $q = 1 - p$

In [15]:
q = 1-p
SE = math.sqrt(p*q/n)
SE

0.06928203230275509

#### 4) Рассчитаем границы доверительного интервала $p\pm t*SE$

In [16]:
ci_low = p - z*SE
ci_hi = p + z*SE
ci_low, ci_hi

(0.46420971191085936, 0.7357902880891406)

Интерпретируем: есть 95% вероятность того, что истинная доля желающих купить iPhone SE находится в интервале от 0.46 до 0.74. Можно вместо доли и проценты использовать - людям это более понятно.

Теперь нужно провести эксперименты. В данном случае опрос одного человека - это испытание Бернулии, т.е. да или нет с фиксированной вероятностью. Давайте представим, что истинная доля желающих купить iPhone SE действительно равна 0.6. Создадим функцию, которая будет проводить эксперименты для распределения Бернулли. Для этого нужно лишь изменить формулу.

In [17]:
def test_conf_bern(dist,alpha,n,num_exp=10000):
    
    z = abs(stats.norm.ppf(alpha/2))
    mu = dist.mean()
    
    result_norm = np.zeros(num_exp)
    
    for exp in range(num_exp):
    
        sample = dist.rvs(n)
        p = sample.mean()
        q = 1 - p
        SE = math.sqrt((p*q)/n)
        
        ci_low_norm = p - z*SE
        ci_hi_norm = p + z*SE
        
        if ci_low_norm <= mu <= ci_hi_norm:
            result_norm[exp] = 1
        
    return result_norm.mean()

Создадим наше распределение Бернулли и протестируем доверительные интервалы.

In [18]:
iphone_bern = stats.bernoulli(0.6)
test_conf_bern(iphone_bern,0.05,50)

0.94530000000000003

Все работает, отлично! Но почему мы вообще используем z-статистику в случае с долей? А все потому, что мы в данном случае апроксимируем биномиальное распределение с помощью нормального. Стоп. Но откуда взялось биномиальное распределение? Биномиальное распределение - это распределение количества успехов в последовательности из независимых случайных экспериментов или испытаний Бернулли. В принципе, можно было даже не писать новую функцию, а просто сделать биномиальное распределение и подставить его в одну из уже написанных функций.

In [19]:
iphone_binom = stats.binom(50,0.6)
test_conf_norm_s(iphone_binom,0.05,50)

0.93869999999999998

Важно помнить, что нормальную апроксимацию биномиального распределения можно использовать только при $np >= 5$ и $nq >= 5$. Так же нормальная апроксимация плохо работает в случае с очень маленькими или очень большими долями. Существует еще несколько методов определения доверительных интервалов для пропорций, например доверительный интервал Клоппера-Пирсона. Приведу пример, когда нормальная апроксимация не работает - скажем в опросе про iPhone из 50 человек положительно ответил только один. Построим доверительный интервал использую нормальную апроксимацию. Для этого я воспользуюсь функцией доверительных интервалов из statmodels.

In [20]:
import statsmodels.api as sm
sm.stats.proportion_confint(1,50,alpha=0.05,method='normal')

(-0.018805307081790987, 0.058805307081790992)

Доля не может быть отрицательной! В этом случае нормальную апроксимацию использовать нельзя. Теперь рассчитаем с помощью Клоппера-Пирсона.

In [21]:
sm.stats.proportion_confint(1,50,alpha=0.05,method='beta')

(0.00050622798304082901, 0.10646954571149991)

Совсем другой доверительный интервал, и в данном случае лучше использовать его. А beta там потому, что в основе этого метода лежит бета-распределение. Ну и в завершение, проведем так же эксперименты. Создадим распределение с маленькой долей и посмотрим, как сработает нормальная апроксимация.

In [22]:
small_proportion = stats.bernoulli(0.1)
test_conf_bern(small_proportion,0.05,50)

0.88460000000000005

Как видно результат совсем неудовлетворительный. Напишем функцию, которая будет тестировать доверительный интервал Клоппера-Пирсона.

In [23]:
def test_conf_bern_beta(dist,alpha,n,num_exp=10000):
    
    z = abs(stats.norm.ppf(alpha/2))
    mu = dist.mean()
    
    result_norm = np.zeros(num_exp)
    
    for exp in range(num_exp):
    
        sample = dist.rvs(n)
        success = sample.sum()
        
        ci_low_norm, ci_hi_norm = \
        sm.stats.proportion_confint(success,n,alpha=alpha,method='beta')
        
        if ci_low_norm <= mu <= ci_hi_norm:
            result_norm[exp] = 1
        
    return result_norm.mean()

In [24]:
test_conf_bern_beta(small_proportion,0.05,50)

0.97099999999999997

Результат гораздо лучше! На этом пока все.